In [ ]:
import spacy 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt 
import spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

In [ ]:
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Downloads/opiates_submissions2.json")
submissions2 = submissions2.head(3000)


submissions3 = pd.read_json("/Users/ritikabrahmadesam/Downloads/opiates_submissions3.json")
submissions3 = submissions3.head(3000)

### This analysis aims to track the frequencies of certain words over time

In [ ]:
#create a column with real datetimes
submissions2['created_utc'] = submissions2['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date).strftime('%m-%d-%Y %H:%M:%S'))
submissions2 = submissions2.sort_values(by='created_utc', ascending=True)

In [ ]:
#remove the stop words and punctionationfrom a line of text
def remove_stops(text):
    nlp = spacy.load("en_core_web_sm")
    excluded_tags = {"VERB", "ADJ", "ADV", "ADP", "PROPN"}
    words = [word.lower() for word in text.split() if word.lower() not in sw_spacy]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    new_text = " ".join(words)
    for elem in punc:
        new_text = new_text.replace(elem, "")
    doc = nlp(new_text)
    for word in doc:
        print(f'Text: {word.text} Part-of-speech: {word.pos_}')
    return new_text


In [39]:
#clean the text by removing stop words, punctuation, and empty entries
submissions2['cleaned_text'] = submissions2['selftext'].apply(lambda b: remove_stops(b))
submissions2 = submissions2.loc[submissions2['cleaned_text']!='',:]

Text: i Part-of-speech: PRON
Text: m Part-of-speech: AUX
Text: wondering Part-of-speech: VERB
Text: insomniac Part-of-speech: ADV
Text: desirable Part-of-speech: ADJ
Text: sleep Part-of-speech: NOUN
Text: effect Part-of-speech: NOUN
Text: strength Part-of-speech: NOUN
Text: lets Part-of-speech: VERB
Text: oxy Part-of-speech: PROPN
Text: xanax Part-of-speech: PROPN
Text: wound Part-of-speech: PROPN
Text: effect Part-of-speech: NOUN
Text: potency Part-of-speech: NOUN
Text: it Part-of-speech: PRON
Text: faking Part-of-speech: VERB
Text: injury Part-of-speech: NOUN
Text: completely Part-of-speech: ADV
Text: order Part-of-speech: NOUN
Text: script Part-of-speech: NOUN
Text: i Part-of-speech: PRON
Text: ve Part-of-speech: AUX
Text: reading Part-of-speech: VERB
Text: complex Part-of-speech: ADJ
Text: regional Part-of-speech: ADJ
Text: pain Part-of-speech: NOUN
Text: syndrome Part-of-speech: NOUN
Text: do Part-of-speech: AUX
Text: nt Part-of-speech: PART
Text: think Part-of-speech: VERB
Text: 

In [ ]:
from collections import Counter
Counter(" ".join(submissions2["cleaned_text"]).split()).most_common(100)